In [ ]:
import pandas as pd
import os
import datetime
from zoneinfo import ZoneInfo
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.cm as cm


tracked_modules = [name for name in os.listdir("C:\\Users\\zhewe\\Documents\\Data\\date_collected\\") if name.startswith('x') and name.endswith('csv')] # 


plt.close('all')
fig, ax = plt.subplots(figsize=(8, 4))

# Set up colormap
cmap = cm.get_cmap('coolwarm')
norm = mcolors.Normalize(vmin=0, vmax=len(tracked_modules) - 1)  # Normalize indices

# Get today's date
today_date = datetime.datetime.now().strftime('%Y%m%d')

for i, module in enumerate(tracked_modules):
    # Read CSV file
    monitoring = pd.read_csv(os.path.join("C:\\Users\\zhewe\\Documents\\Data\\date_collected", module))

    # Convert timestamps to PST
    monitoring['Datetime PST'] = [datetime.datetime.utcfromtimestamp(ts)
                                  .replace(tzinfo=ZoneInfo("UTC"))
                                  .astimezone(ZoneInfo("America/Los_Angeles"))
                                  for ts in monitoring['Time']]

    # Save to CSV with the format: label_YYYYMMDD.csv
    label = module.split('_')[0]
    output_filename = f"{label}_{today_date}.csv"
    output_path = os.path.join("C:\\Users\\zhewe\\Documents\\Data", output_filename)
    
    # Save relevant columns
    monitoring[['Time', 'Datetime PST', 'Calc Current (A)']].to_csv(output_path, index=False)
    
    # Assign a color for plotting
    color = cmap(norm(i))
    ax.scatter(monitoring['Datetime PST'], monitoring['Calc Current (A)'], s=5, label=label, alpha=0.8, color=color)

# Format x-axis as PST
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M', tz=ZoneInfo("America/Los_Angeles")))
ax.xaxis.set_major_locator(mdates.AutoDateLocator())

# Plot settings

plt.xticks(rotation=15)
plt.ylabel('Current (A)')
plt.xlabel('Date (PST)')

plt.title('Leakage Current vs Time')

plt.legend(ncols=2)
plt.show()

print("CSV files saved successfully!")
